**DEB**

**ANALYSE DU PROBLEME**
---

Problème de classification (28 classes) avec évaluation "sur mesure" (composite) qui tient compte de la performance et de l'équité du modèle<br><br>. 

Jeu de donnée de 25.000 personnes, après voir lancé la baseline, j'ai entrainé un **adversarial NN**, qui permettait de gagner un peu d'équité (passage de 80% à 85% sur le TRP gap) mais a entrainé un éffondrement de l'accuracy (de 65% à 35%). après avoir cherché à personaliser la fonction de perte (avec difficulté pour entrainer le modele), je suis repassée à une approche plus progressive, sur la régression logistique

**regression logisitique (baseline) => 3 problemes**<br>
1. ***l'échantillonage*** : Le "train_test_split" ne tient pas compte de S, nous ne savons pas si l'attribut protégé (H/F) est bien réparti entre X_train et X_test, cela n'est pas optimum pour l'apprentissage car certaines classes contiennent peu de points. Il peut aussi etre plus pertinent de fixer les hyperparametre grace à X_train, et d'entrainer le modele sur tout l'échantillon (avec les memes hyperparametres) avant de prédire X_test_true et de soumettre<br>
=> nous réaliserons des train_test_split en tenant compte de X et Y grace à Y56 = Y + 28*S <br>
=> nous ferons des soumissions double (modele appris sur X_train et aussi X) pour le datachallenge, pour améliorer les qualités prédicitive du modèles <br><br>
2. la fonction d'évaluation est une ***moyenne de scores calculé par classes***, chaque classe a donc le meme poids dans le score final. Or, l'apprentissage est fait pour optimser l'accuracy de l'échantillon (chaque élément à le meme poids). Comme la distribution de l'échatillon est très mal équilibrée entre les classes (93 à 8.285 personnes), le modèle ne peut apprendre correctement<br>
=> nous essaierons pour la forme del a data augmentation, mais sans conviction compte tenu de l'embedding sémantique fait avec BERT<br>
=> nous utiliserons une correction de la fonction de pertes pour y inclure le poids de chaque classe (Y et Y56)<br>
=> nous essaierons du contrastive learning pour obtenir un espace de réprésentation adéquat<br><br>
2. ***l'apprentissage n'est pas optimal (alignement et hyperparametres à améliorer)***. Il manque la régularization et du cross validation. L'apprentissage ,'est pas aligné avec "score final": le modele fait pour optimiser l'accuracy (la précision), or la fonction d'évaluation est une moyenne entre le F1 score (moyenne harmonique de la précision et du rappel) et une métrique de fairness. Le F1 score pénalise les écarts importants entre la précision et le rappel. Le score<br>
=> nous adapterons le fonctions de pertes et/ou l'architecture des modèles intégrer au mieux notrescore final
=> nous ajouterons systematiquement des termes de régularisations et de cross validation (stratified k-fold cross validation si possibl)<br> 

**METHODOLOGIE**
---

**Nous allons procéder en 4 temps** :

**1/ optimiser la régression logisitique**<br>
<br>
ajouter double prédiction sur S et nonS<br><br>

**2/ débiaiser l'échantillon**<br> 


**3/ personaliser la foncition de perts**

**4/ explorer modeles précis à la frontiere**<br>
KNN
SVM linéaire et gaussien
1 versus all sur classe mal prédites

**5/ regression linéaire des modèles**
échantilloner X par rapport à Y et S** traiter le probleme 1 et 2, en créant un label à 56 classes (2 x 28) pour distinguer chaque label i en fonction de s'il est protégé ou non. Nous augmenterons la taille de l'échantillon (avec 56 labels) pour créer X_XL et Y_XL pour avoir le meme nombre d'items pour chacune des 56 classes et un jeu avec le meme nombre d'instance par classe, permettant d'aligner les scores moyens des classes avec la moyenne du nouveau sample "XL" décuplé (passage de 27.749 lignes à 255.304 lignes ). Nous programmerons la régression logisitique avec pytorch pour accélérer les calculs.

2/ nous **ajusterons ensuite la fonction de pertes**, pour qu'elle tienne compte au mieux de la fonction d'évaluation, qui est composée de 2 éléments : le F1 score et le 1-equal ooprtunity gap<br>
- le 'macro f score' (F1 score) n'est pas dérivable (à valider), nous l'approcherons donc par accuracy et le recall<br>
- true positive gap : 1 - TRP_GAP est calculé comme 
A noter que le F1 score n'est pas dérivable. However, we can approximate the F1 score in a differentiable manner by using the ***Sørensen–Dice coefficient***, which is closely related to the F1 score and is differentiable. The Sørensen–Dice coefficient is defined as \(2 * \frac{precision * recall}{precision + recall}\), which is equivalent to the F1 score formula. For a differentiable approximation, we can use soft versions of precision and recall.
<br><br>

3/ Nous explorerons **différents modèles de classification** K-means, SVM, random forest tree<br><br>

4/ Avant de mener cette approche j'avais essayer 



**A) Import, loading data, eval functions**
---

In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.datasets import make_classification
from sklearn.metrics import make_scorer #, f1_score

#from Data_challenge_fairness_2024.evaluator import *
from evaluator import *


In [6]:

path_model = 'FINAL_models/'  # must finish by '/'
path_Y_pred_true = 'FINAL_Y_pred_true/'  # must finish by '/'

# with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)
# with open(path_model + name + '.pkl', 'rb') as f: model = pickle.load(f)

def get_final_score(Y_pred, Y, S):
    eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
    final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2
    return final_score
    
    
def get_scores(Y_pred, Y, S):
    accuracy= accuracy_score(Y, Y_pred)  # Y_test are your original test labels
    print(f"Accuracy on transformed test data: {accuracy}")
    eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
    final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2
    macro_f1 = eval_scores['macro_fscore']
    inv_macro_gap = 1-eval_scores['TPR_GAP']

    #print results
    print('final score :',final_score)
    print('macro_f1    :',eval_scores['macro_fscore'])
    print('inv_macro_gap',1-eval_scores['TPR_GAP'])
    
    return accuracy, final_score, macro_f1, inv_macro_gap

In [8]:
#####################################################
# Load pickle file and convert to numpy array
#####################################################

with open('data-challenge-student.pickle', 'rb') as handle:
    # dat = pickle.load(handle)
    dat = pd.read_pickle(handle)

#Check keys()
print(dat.keys())
X = dat['X_train']
Y = dat['Y']
S = dat['S_train']

X_test_true = dat['X_test']
S_test_true = dat['S_test']

Y56= Y + 28*S
#X, X_test,Y,S, S_test = dat[1]

print(X.shape,Y.shape,S.shape,X_test_true.shape,S_test_true.shape)

X_train, X_test, Y_train, Y_test, S_train, S_test = train_test_split(X, Y, S, test_size=0.3, random_state=42)

path_model = 'FINAL_models/'
path_Y_pred_true = 'FINAL_Y_pred_true/'

dict_keys(['X_train', 'X_test', 'Y', 'S_train', 'S_test'])
(27749, 768) (27749,) (27749,) (11893, 768) (11893,)


In [15]:
#################################################################
#   DEBIASING X : CREATION OF ORTHOGONAL PROJECTION 
################################################################

#------------------------------------------------------------------------
# Bias estimation in dataset
#------------------------------------------------------------------------

# Calculate average embedding for group sensitive (S=1) and non sensitive (S=0)
X_sensitive = X[S==1]
X_sensitive_mean=X_sensitive.mean()

X_non_sensitive = X[S!=1]
X_non_sensitive_mean=X_non_sensitive.mean()

bias = X_sensitive_mean-X_non_sensitive_mean

#------------------------------------------------------------------------
# function to debiase data
#------------------------------------------------------------------------

def remove_bias(dataset, bias):
    ''' orthogonal projection of dataset / bias'''
    # Compute the dot product of each row of the dataset with diff
    dot_products = np.dot(dataset, bias)
    
    # Compute the magnitude of bias
    bias_magnitude_squared = np.dot(bias, bias)
    
    # Compute the projection of each row onto diff
    projection = np.outer(dot_products / bias_magnitude_squared, bias)
    
    # Subtract the projection from the dataset
    debiased_dataset = dataset - projection
    
    return debiased_dataset

#------------------------------------------------------------------------
# Debiased X and X_test_true
#------------------------------------------------------------------------

debiased_X = remove_bias(X, bias)
debiased_X_test_true = remove_bias(X_test_true, bias)


**LOGISTIC REGRESSION MODELS**
---
<br>
0_Baseline : logistic regression (on X_train)<br><br>
1_Optimised : optimized logistic regression (on X + hyperparameters+ stratified k-fold cross validation)<br><br>
2_Stratified K fold Logistic Regression
3_Orthogonal debiasing of X on model 2 (Optimised Logisitic regression with custom loss function) <br><br>
4_Orthogonal debiasing of X on MAN/WOMAN embedding difference  on model 2 (Optimised Logisitic regression with custom loss function) <br><br>
5_Custom loss fonctin on Optimised Logisitic regression (56 classes with Y56= Y + 28*S) 

In [ ]:
RESULTS = pd.DataFrame(columns=['Model','Accuracy','final score','Fscore macro','1 - TPR gap'])
RESULTS.loc[0]=['baseline (Reglog28)',accuracy_0,final_score_0, eval_scores_0['macro_fscore'],1-eval_scores_0['TPR_GAP']]
RESULTS.loc[2]=['RegLog28 + YxS',accuracy_1,final_score_1, eval_scores_1['macro_fscore'],1-eval_scores_1['TPR_GAP']]
RESULTS.loc[4]=['RegLog28 + XL',accuracy_2,final_score_2, eval_scores_2['macro_fscore'],1-eval_scores_2['TPR_GAP']]
RESULTS.loc[5]=['RegLog28 + YxS + XL',accuracy_3,final_score_3, eval_scores_3['macro_fscore'],1-eval_scores_3['TPR_GAP']]
RESULTS.loc[6]=['RegLog56 + YxS + XL',accuracy_4,final_score_4, eval_scores_4['macro_fscore'],1-eval_scores_4['TPR_GAP']]
RESULTS.loc[3]=['RegLog56 + YxS',accuracy_5,final_score_5, eval_scores_5['macro_fscore'],1-eval_scores_5['TPR_GAP']]
RESULTS.loc[1]=['baseline seed=1 (Reglog28)',accuracy_0_1,final_score_0_1, eval_scores_0_1['macro_fscore'],1-eval_scores_0_1['TPR_GAP']]
RESULTS = RESULTS.sort_index()
RESULTS

with open('RESULTS_10-03-2024.pkl', 'wb') as f:
   pickle.dump(RESULTS, f)

#RESULTS =  pd.read_pickle('RESULTS_10-03-2024.pkl')

**LOGISTIC REGRESSION**
---

In [4]:
###############################################
# 0. LOGISTIC REGRESSION - BASELINE 
###############################################

name ='0_Reglog_baseline' # changer clf_i

# Refresh training data
X_train, X_test, Y_train, Y_test, S_train, S_test = train_test_split(X, Y, S, test_size=0.3, random_state=42)

# training (or load)logistic model
#clf_0 = LogisticRegression(random_state=0, max_iter=5000,verbose=1).fit(X_train, Y_train)
with open(path_model + name + '.pkl', 'rb') as f: clf_0 = pickle.load(f)
model = clf_0 
  
# predicting and assessing
Y_pred = model.predict(X_test)
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
Y_pred_true = model.predict(X_test_true)
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)

Accuracy on transformed test data: 0.7638438438438439
final score : 0.7365033273594812
macro_f1    : 0.669337208333607
inv_macro_gap 0.8036694463853555


In [5]:
##################################################
# 1. LOGISTIC REGRESSION - BASELINE OPTIMISED
# (no validation set - training on all X)
##################################################

name ='1_Reglog_optimised' # changer clf_i

# Refresh training data
# TRAINED ON ALL X

# training (or load)logistic model
# added regularisation 'l2' with coeff 0.2 (C) and early stopping with tol = 0.0001
# clf_1 = LogisticRegression(random_state=42, max_iter=5000,verbose=1,penalty='l2', C=0.2, tol=0.0001).fit(X, Y)
with open(path_model + name + '.pkl', 'rb') as f: clf_1 = pickle.load(f)
model = clf_1

# predicting and assessing
Y_pred = model.predict(X_test)
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
Y_pred_true = model.predict(X_test_true)
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        21532     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.24653D+04    |proj g|=  4.11128D+04


 This problem is unconstrained.



At iterate   50    f=  2.05318D+04    |proj g|=  8.07961D+02

At iterate  100    f=  1.73620D+04    |proj g|=  6.71832D+02

At iterate  150    f=  1.61758D+04    |proj g|=  1.03107D+02

At iterate  200    f=  1.57705D+04    |proj g|=  1.24845D+02

At iterate  250    f=  1.56370D+04    |proj g|=  3.30421D+01

At iterate  300    f=  1.55878D+04    |proj g|=  2.25705D+01

At iterate  350    f=  1.55716D+04    |proj g|=  3.01638D+01

At iterate  400    f=  1.55661D+04    |proj g|=  1.65593D+01

At iterate  450    f=  1.55641D+04    |proj g|=  1.12923D+01

At iterate  500    f=  1.55632D+04    |proj g|=  3.90256D+00

At iterate  550    f=  1.55628D+04    |proj g|=  6.05409D+00

At iterate  600    f=  1.55624D+04    |proj g|=  4.51413D+00

At iterate  650    f=  1.55619D+04    |proj g|=  1.26648D+01

At iterate  700    f=  1.55610D+04    |proj g|=  7.07335D+00

At iterate  750    f=  1.55595D+04    |proj g|=  1.62580D+01

At iterate  800    f=  1.55577D+04    |proj g|=  4.04886D+00

At iter

In [23]:
##################################################
# 2. LOGISTIC REGRESSION - STRATIFIED K-FOLD
# on optimized Log. Reg. regularisatio, early stoppping
# (no validation set - training on all X)
##################################################

name ='2_Reglog_StratKFold' # changer clf_i

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score

# Refresh training data
# TRAINED ON ALL X

# training (or load)logistic model
# added regularisation 'l2' with coeff 0.2 (C) and early stopping with tol = 0.0001
clf_2 = LogisticRegression(random_state=42, max_iter=5000,verbose=0,penalty='l2', C=0.2, tol=0.0001)

# Préparer la validation croisée k-fold stratifiée
cv = StratifiedKFold(n_splits=10)
scores = cross_val_score(clf_2, X, Y, scoring='accuracy', cv=cv, n_jobs=-1)
clf_2.fit(X, Y)
model = clf_2
# with open(path_model + name + '.pkl', 'rb') as f: clf_1 = pickle.load(f)

# predicting and assessing
Y_pred = model.predict(X_test)
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
Y_pred_true = model.predict(X_test_true)
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)


Accuracy on transformed test data: 0.856096096096096
final score : 0.8207123104282983
macro_f1    : 0.8407599159632768
inv_macro_gap 0.8006647048933198


In [10]:
##########################################################################
# 3. DEBIASING WITH ORTHOGONAL PROJECTION + OPTIMISED MODEL 
# (using the vector of the average difference of embedding between groups 
# to debiase representation of X, by operating an orthogonal projection)
##########################################################################

name ='3_Reglog_Optimized_orthogonal' # changer clf_i

#---------------------------------------------------------------------
# Debiasing X (Projection of X orthogonally to bias) 
#---------------------------------------------------------------------

# projection in new representation space
debiased_X = remove_bias(X, bias)

#---------------------------------------------------------------------
# TRAINING MODEL AND PREDICTING
#---------------------------------------------------------------------

# clf_3 = LogisticRegression(random_state=0, max_iter=5000,verbose=0,penalty='l2', C=0.2, tol=0.0001).fit(debiased_X, Y)
with open(path_model + name + '.pkl', 'rb') as f: clf_3 = pickle.load(f)
model = clf_3

# predicting and assessing
Y_pred = model.predict(X_test)
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
debiased_X_test_true = remove_bias(X_test_true, bias)  # debiasing
Y_pred_true = model.predict(debiased_X_test_true)
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)


#
# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)

Accuracy on transformed test data: 0.8545345345345345
final score : 0.8290222283482765
macro_f1    : 0.8381637333117026
inv_macro_gap 0.8198807233848505


/home/anaele/myenv/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.3.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [49]:
##########################################################################
# 4. APPROCHE DUALE : MODEL HOMME + MODEL FEMME (DEBIASED X)
# + Debiasing with orthogonal projection + Optimised Logisitic Regression
##########################################################################

name ='4_Reglog_DUAL_orthogonal_FxH' # changer clf_i

# ----------------------------------------------------
# PREPARATION DES DONNEES
#-----------------------------------------------------

# generating samples for Woman (S=1) and Men (S=0)
X_F , X_H = X[S==1] , X[S==0]
Y_F , Y_H = Y[S==1] , Y[S==0]
S_F , S_H = S[S==1] , S[S==0]
X_test_true_F , X_test_true_H = X_test_true[S_test_true==1] , X_test_true[S_test_true==0]
S_test_true_F , S_test_true_H = S_test_true[S_test_true==1] , S_test_true[S_test_true==0]
# debiasing X and X_test_true

debiased_X = remove_bias(X,bias)
debiased_X_test = remove_bias(X_test,bias)
debiased_X_test_true = remove_bias(X_test_true, bias)

debiased_X_F , debiased_X_H = debiased_X[S==1] , debiased_X[S==0]
debiased_X_test_F , debiased_X_test_H = debiased_X_test[S_test==1] , debiased_X_test[S_test==0]
debiased_X_test_true_F , debiased_X_test_true_H = debiased_X_test_true[S_test_true==1] , debiased_X_test_true[S_test_true==0]
print(X_F.shape,X_H.shape)
print(Y_F.shape,Y_H.shape)
print(S_F.shape,S_H.shape)

# ----------------------------------------------------
# ENTRAINEMENT DES MODELES
#-----------------------------------------------------

'''# Prepare Logistic Regression models
# model 4 # model_X = LogisticRegression(random_state=0, max_iter=5000,penalty='l2', C=0.2, tol=0.0001).fit(X,Y)
model_F = LogisticRegression(random_state=0, max_iter=5000,penalty='l2', C=0.2, tol=0.0001)
model_H = LogisticRegression(random_state=0, max_iter=5000, penalty='l2', C=0.2, tol=0.0001)

# Train the models
model_F.fit(debiased_X_F, Y_F)
model_H.fit(debiased_X_H, Y_H)'''
with open(path_model + name + '.pkl', 'rb') as f: clf_3 = pickle.load(f)
model = clf_3

# predicting and assessing on test
Y_pred_F = model_F.predict(debiased_X_test_F)
Y_pred_H = model_H.predict(debiased_X_test_H)
# compute final result depending on S_test_true
Y_pred_all = np.zeros(len(Y_test))
Y_pred_all[ S_test == 1 ] = Y_pred_F
Y_pred_all[ S_test == 0 ] = Y_pred_H
# Y_pred_F * S_test + Y_pred_H * (1-S_test)
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred_all,Y_test,S_test)

# Predict model on test_true
Y_pred_true_F = model_F.predict(debiased_X_test_true_F)
Y_pred_true_H = model_H.predict(debiased_X_test_true_H)
# compute final result depending on S_test_true
Y_pred_true_all = np.zeros(len(X_test_true))
Y_pred_true_all[ S_test_true == 1 ] = Y_pred_true_F
Y_pred_true_all[ S_test_true == 0 ] = Y_pred_true_H
# Y_pred_true_all = Y_pred_true_F * S_test_true + Y_pred_true_H * (1-S_test_true)

results=pd.DataFrame(Y_pred_true_all, columns= ['score'])
results['score'] = results['score'].astype(int)
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)
#results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

#
# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)
with open(path_model + '4_Reglog_DUAL_orthogonal_F' + '.pkl', 'wb') as f: pickle.dump(model_F, f)
with open(path_model + '4_Reglog_DUAL_orthogonal_H' + '.pkl', 'wb') as f: pickle.dump(model_H, f)


(12793, 768) (14956, 768)
(12793,) (14956,)
(12793,) (14956,)
Accuracy on transformed test data: 0.8742342342342342
final score : 0.8042258252858332
macro_f1    : 0.865061187492607
inv_macro_gap 0.7433904630790595


In [12]:
'''##########################################################################
# 4. BERT (WOMAN-MAN) DEBIASING WITH ORTHOGONAL PROJECTION + MODEL 2 (k-fold)
# (using the vector of the difference of embedding between WOMAN and MAN
# to debiase representation of X, by operating an orthogonal projection)
# !!!! uncertainty about model of BERT used for the training !!!!
##########################################################################

name ='4_Reglog_StratKFold_orthogonalBERT' # changer clf_i

#------------------------------------------------------------------------
# GET MAN AND WOMAN EMBEDDING IN BERT
#-----------------------------------------------------------------------

from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import tokenizers as tk
from tqdm import tqdm

# Charger le tokenizer et modèle de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

# Tokenize "man" and "woman"
tokens_man = tokenizer.encode("man", add_special_tokens=True)  # Ajoute les tokens spéciaux
tokens_woman = tokenizer.encode("woman", add_special_tokens=True)

print(tokens_man)
print(tokens_woman)

# transforme en tensor de batch (meme si un seul mot)

input_ids_man = tf.constant(tokens_man)[None, :]  # Ajoute une dimension de batch
input_ids_woman = tf.constant(tokens_woman)[None, :]  # Ajoute une dimension de batch

# Obtenir les embeddings
outputs_man = model(input_ids_man)
outputs_woman = model(input_ids_woman)

# Les embeddings du dernier layer pour le premier token ('[CLS]' par défaut)
embedding_man = outputs_man.last_hidden_state[0][0]
embedding_woman = outputs_woman.last_hidden_state[0][0]
bias_BERT = embedding_woman - embedding_man
bias_BERT = np.array (bias_BERT)
print(embedding_man.shape)  # Doit être (768,)
print(embedding_woman.shape)  # Doit être (768,)
print(bias_BERT.shape)  # Doit être (768,)

#---------------------------------------------------------------------
# Debiasing X (Projection of X orthogonally to bias) 
#---------------------------------------------------------------------

# projection in new representation space
debiased_X_BERT = remove_bias(X, bias_BERT)

#----------------------------------------------------------------------
# TRAINING MODEL AND PREDICTING
#------------------------------------------------------------------------

clf_4 = LogisticRegression(random_state=42, max_iter=5000,verbose=1,penalty='l2', C=0.2, tol=0.0001)

# Préparer la procédure de validation croisée k-fold stratifiée
cv = StratifiedKFold(n_splits=10)
#custom_scorer = make_scorer(final_score_S)
scores = cross_val_score(clf_4, debiased_X_BERT, Y, cv=cv, n_jobs=-1)
clf_4.fit(debiased_X_BERT, Y)
model = clf_4
# with open(path_model + name + '.pkl', 'rb') as f: clf_3 = pickle.load(f)

# predicting and assessing
Y_pred = model.predict(X_test)
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
modified_X_test_true = remove_info(X_test_true, bias_BERT)  # debiasing
Y_pred_true = model.predict(modified_X_test_true)
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)'''

'##########################################################################\n# 4. BERT (WOMAN-MAN) DEBIASING WITH ORTHOGONAL PROJECTION + MODEL 2 (k-fold)\n# (using the vector of the difference of embedding between WOMAN and MAN\n# to debiase representation of X, by operating an orthogonal projection)\n# !!!! uncertainty about model of BERT used for the training !!!!\n##########################################################################\n\nname =\'4_Reglog_StratKFold_orthogonalBERT\' # changer clf_i\n\n#------------------------------------------------------------------------\n# GET MAN AND WOMAN EMBEDDING IN BERT\n#-----------------------------------------------------------------------\n\nfrom transformers import BertTokenizer, TFBertModel\nimport tensorflow as tf\nimport tokenizers as tk\nfrom tqdm import tqdm\n\n# Charger le tokenizer et modèle de BERT\ntokenizer = BertTokenizer.from_pretrained(\'bert-base-multilingual-cased\')\nmodel = TFBertModel.from_pretrained(\'bert-base-mu

**MODEL 6 - ADPAPTING LOSS FUNCTION IN LOGISTIC REGRESSION (PYTORCH)**
---
1. Functions for Custom loss function (re-written in pytorch)
2. Functions for Stratified K-fold and training batches
3. Running Model 6
4. Optimising Model 6 hyperparameters

In [13]:
''' PROBLEME SUR FINAL SCORE QUI DEPEND DE S, OR SICKIT LEARN NE GERE
PAS S DANS LA CROSS VALIDATION, LA LOSS FUNCTION PERSONALISEE NE PEUT 
ETRE CALCULEE  =======> CODAGE SUR PYTORCH

##########################################################################
# 5. CUSTOM LOSS FUNCTION (Using stratified K + custom_scorer)
# + Debiasing with orthogonal projection + Optimised Logisitic Regression
##########################################################################

name ='5_Reglog_StratKFold1_Orthogonal_CustomLoss' # changer clf_i

#---------------------------------------------------------------------
# Debiasing X (Projection of X orthogonally to bias) 
#---------------------------------------------------------------------

# projection in new representation space
debiased_X = remove_bias(X, bias)

#------------------------------------------------------------------------
# CREATION OF CUSTOM LOSS FUNCTION (FINAL SCORE) 
# -----------------------------------------------------------------------

#It is not possible to split S with the stratified k-fold implementation 
# in sickit learn. We will use Y56 = Y + 28*S to train logistic model
# and derive from Y56 the value of Y = Y56 % 28 and S = Y56//28

def final_score_S(Y,Y_pred):
    # wrapper function to include S
    # Note order : custom scorer expects (y_true, y_pred) as inputs
    
    return get_final_score(Y_pred,Y,S)

custom_scorer = make_scorer(final_score_S)

#---------------------------------------------------------------------
# TRAINING MODEL AND PREDICTING
#---------------------------------------------------------------------

clf_5 = LogisticRegression(random_state=0, max_iter=5000,verbose=0,penalty='l2', C=0.2, tol=0.0001)

# Préparer la procédure de validation croisée k-fold stratifiée
cv = StratifiedKFold(n_splits=1)
scores = cross_val_score(clf_3, debiased_X, Y, scoring=custom_scorer ,cv=cv, n_jobs=-1)
clf_5.fit(debiased_X, Y56)
model = clf_5
# with open(path_model + name + '.pkl', 'rb') as f: clf_5 = pickle.load(f)

# predicting and assessing
Y56_pred = model.predict(X_test)
#S_pred = Y56//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)
Y_pred = Y56_pred % 28  # reste (original Y)   ex 33% 28 = classe 5
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
modified_X_test_true = remove_info(X_test_true, bias)  # debiasing
Y56_pred_true = model.predict(modified_X_test_true)
Y_pred_true = Y56_pred_true % 28  # reste (original Y)   ex 33% 28 = classe 5
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)'''

' PROBLEME SUR FINAL SCORE QUI DEPEND DE S, OR SICKIT LEARN NE GERE\nPAS S DANS LA CROSS VALIDATION, LA LOSS FUNCTION PERSONALISEE NE PEUT \nETRE CALCULEE  =======> CODAGE SUR PYTORCH\n\n##########################################################################\n# 5. CUSTOM LOSS FUNCTION (Using stratified K + custom_scorer)\n# + Debiasing with orthogonal projection + Optimised Logisitic Regression\n##########################################################################\n\nname =\'5_Reglog_StratKFold1_Orthogonal_CustomLoss\' # changer clf_i\n\n#---------------------------------------------------------------------\n# Debiasing X (Projection of X orthogonally to bias) \n#---------------------------------------------------------------------\n\n# projection in new representation space\ndebiased_X = remove_bias(X, bias)\n\n#------------------------------------------------------------------------\n# CREATION OF CUSTOM LOSS FUNCTION (FINAL SCORE) \n# ----------------------------------------

In [32]:
##########################################################################
# 5. CUSTOM LOSS FUNCTION WITH 56 CLASS CLASSIFIER (USING Y56 = Y + S*28)
# + Debiasing with orthogonal projection + Optimised Logisitic Regression
##########################################################################

name ='5_Reglog56_StratKFold_Orthogonal_CustomLoss' # changer clf_i

#---------------------------------------------------------------------
# Debiasing X (Projection of X orthogonally to bias) 
#---------------------------------------------------------------------

# projection in new representation space
debiased_X = remove_bias(X, bias)

#------------------------------------------------------------------------
# CREATION OF CUSTOM LOSS FUNCTION (FINAL SCORE) 
# -----------------------------------------------------------------------

#It is not possible to split S with the stratified k-fold implementation 
# in sickit learn. We will use Y56 = Y + 28*S to train logistic model
# and derive from Y56 the value of Y = Y56 % 28 and S = Y56//28

def final_score_S(Y56,Y_pred):
    '''custom scorer expects (y_true, y_pred) as inputs
    Inputs : Y56 and Y pred
    Outputs : final score on 28 classes with Y56 trick (S=Y56//28 and Y=Y56%28)'''
    S = Y56//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)
    Y = Y56 % 28  # reste (original Y)   ex 33% 28 = classe 5 
    return get_final_score(Y_pred,Y,S)

custom_scorer = make_scorer(final_score_S)

#---------------------------------------------------------------------
# TRAINING MODEL AND PREDICTING
#---------------------------------------------------------------------

'''BlockingIOErrorclf_5 = LogisticRegression(random_state=42, max_iter=5000,verbose=0,penalty='l2', C=0.2, tol=0.0001)

# Préparer la procédure de validation croisée k-fold stratifiée
cv = StratifiedKFold(n_splits=10)
scores = cross_val_score(clf_5, debiased_X, Y56, scoring=custom_scorer ,cv=cv, n_jobs=-1)
clf_5.fit(debiased_X, Y56)'''
# with open(path_model + name + '.pkl', 'rb') as f: clf_5 = pickle.load(f)
model = clf_5

# predicting and assessing
Y56_pred = model.predict(X_test)
#S_pred = Y56//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)
Y_pred = Y56_pred % 28  # reste (original Y)   ex 33% 28 = classe 5
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)


# predict X_test_true and save
modified_X_test_true = remove_bias(X_test_true, bias)  # debiasing
Y56_pred_true = model.predict(modified_X_test_true)
Y_pred_true = Y56_pred_true % 28  # reste (original Y)   ex 33% 28 = classe 5
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)

In [33]:
##########################################################################
# 6. CUSTOM LOSS FUNCTION IN PYTORCH (CLASSIFIER ON 28 ORIGINAL CLASSES)
# + Debiasing with orthogonal projection + Optimised Logisitic Regression
##########################################################################

name ='5_Reglog56_StratKFold_Orthogonal_CustomLoss' # changer clf_i

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader


#########################################################################
#     CUSTOM LOSS FUNCTION AND EVALUATION FUNCTIONS
#         (RE-WRITTEN FOR PYTORCH)
#
#   soft_f1_loss
#   macro_soft_f1_loss
#   get_macro_f1
#   get_tpr_gap
#   get_macro_tpr_gap
#   soft_final_score_loss
#   get_final_score
#
###########################################################################


def soft_macro_f1_loss(y_true, y_pred):
    """
    Differentiable approximation of the macro F1 score as a loss function.
    Calculates the F1 score for each class independently and then takes the average.
    Inputs :
        y_true must be one hot encoded
    """
    y_pred_one_hot = torch.nn.functional.one_hot(y_pred, num_classes=Y_train.nunique()) if len(y_pred.shape) == 1 else y_pred
    #y_pred_probs = torch.softmax(y_pred_one_hot, dim=1)
    
    tp = torch.sum(y_true * y_pred, dim=0)
    pp = torch.sum(y_pred, dim=0)
    ap = torch.sum(y_true, dim=0)
    
    precision = tp / (pp + 1e-6)
    recall = tp / (ap + 1e-6)
    
    f1_per_class = 2 * (precision * recall) / (precision + recall + 1e-6)
    macro_f1 = torch.mean(f1_per_class)   # Mean to aggregate over all classes
    
    loss = 1 - macro_f1  # Minimizing loss is maximizing macro F1 score
    return loss


def get_macro_f1(y_true, y_pred):
    """
    Calculate the exact macro F1 score for evaluation.
    Input : any format as tensors will be converted to Tensors of true label if dim >1 . Can be :
        - Tensor of probabilities(y_pred_probs) dimension (n,28)
        - Tensor of labels, one hote encoded (y_pred_one_hot) dimension (n,28)
        - Tensor of labels (y_pred_tensor) dimension (n,1)
    Ouput : scalar
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true

    " predict macro f1"
    f1 = f1_score(y_true_labels.cpu().numpy(), y_pred_labels.cpu().numpy(), average='macro')
    return f1

def get_tpr_gap(y_true, y_pred, protected_attribute, class_idx):
    """
    Calculate the TPR gap for a specific class across protected groups.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred_probs: Tensor of predicted probabilities (after softmax).
    - protected_attribute: Tensor indicating group membership for each instance.
    - class_idx: Index of the class for which to calculate the TPR gap.
    
    Returns:
    - TPR gap for the specified class.
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true
    
    # Calculate overall TPR for the current class
    overall_mask = y_true_labels == class_idx
    overall_tpr = torch.sum((y_pred_labels == class_idx) & overall_mask).float() / (torch.sum(overall_mask).float() + 1e-6)
    
    # Initialize list to store TPR for each protected group
    group_tprs = []
    
    # Calculate TPR for each protected group
    for group_val in protected_attribute.unique():
        group_mask = (protected_attribute == group_val) & overall_mask
        group_tpr = torch.sum((y_pred_labels == class_idx) & group_mask).float() / (torch.sum(group_mask).float() + 1e-6)
        group_tprs.append(group_tpr)
    
    # Calculate TPR gap for the current class
    tpr_gaps = torch.abs(torch.tensor(group_tprs) - overall_tpr)
    
    return torch.mean(tpr_gaps)  # Return the mean TPR gap for this class

def get_macro_tpr_gap(y_true, y_pred, protected_attribute):
    """
    Calculate the average TPR gap per class by calling tpr_gap for each class.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred: Tensor of predicted logits (before softmax).
    - protected_attribute: Tensor indicating group membership for each instance.
    
    Returns:
    - Average TPR gap across all classes.
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true
    
    # Initialize list to store TPR gaps for all classes
    class_tpr_gaps = []
    
    # Iterate over each class
    num_classes = len(y_true_labels.unique())
    for class_idx in range(num_classes):
        class_tpr_gap = get_tpr_gap(y_true_labels, y_pred_labels, protected_attribute, class_idx)
        class_tpr_gaps.append(class_tpr_gap)
    
    # Calculate the average TPR gap across all classes
    avg_tpr_gap = torch.mean(torch.stack(class_tpr_gaps))
    
    return avg_tpr_gap


def soft_final_score_loss(y_true, y_pred, protected_attribute):
    """
    Combine soft macro F1 score and TPR gap to create a final evaluation metric.
    """
    soft_macro_f1 = soft_macro_f1_loss(y_true, y_pred)  # Calculate soft macro F1 score
    macro_tpr_gap = get_macro_tpr_gap(y_true, y_pred, protected_attribute)  # Calculate TPR gap
    
    soft_final_score = ( soft_macro_f1 + (1 - macro_tpr_gap) ) / 2
    return soft_final_score

def get_final_score(y_true, y_pred, protected_attribute):
    """
    Combine soft macro F1 score and TPR gap to create a final evaluation metric.
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true

    macro_f1 = get_macro_f1(y_true_labels, y_pred_labels)  # Calculate macro F1 score
    macro_tpr_gap = get_macro_tpr_gap(y_true_labels, y_pred_labels, protected_attribute)  # Calculate macro TPR gap
    
    final_score = (macro_f1 + (1 - macro_tpr_gap)) / 2
    return final_score

#---------------------------------------------------------------------
# TORCH EVALUATION FUNCTIONS
#--------------------------------------------------------------------


def evaluate(Y_pred,Y,S,will_print=1):
    '''returns model accuracy, final score, macro fscore ans TPR gap
    input : 2 np arrays of same dimension
    output : array of 4 values
    '''
    accuracy= accuracy_score(Y, Y_pred)  # Y_test are your original test labels
    print(f"Accuracy on transformed test data: {accuracy}")
    eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
    final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2

    if will_print==1:
        #print results
        print('final score',final_score)
        print('macro_fscore',eval_scores['macro_fscore'])
        print('1-eval_scores[\'TPR_GAP\']',1-eval_scores['TPR_GAP'])
    
    return accuracy, final_score, eval_scores['macro_fscore'],1-eval_scores['TPR_GAP'] , eval_scores , confusion_matrices_eval

# to predict X_test and save to file

def save_Y_pred_tofile(X, model, name): # adapted to torch
    
    # save probabilities for each Xi (dim=28)
    y_pred_probs = model(X)
    probs=pd.DataFrame(y_pred_probs.detach().numpy(), columns= list(range(0,28)))
    file_name_probs = "y_pred_probs/y_pred_probs_"+str(name)+".csv"
    probs.to_csv(file_name_probs, header = None, index = None)

    # save predicted labels for each Xi (dim=1)
    y_pred = torch.argmax(y_pred_probs, dim=1)
    results=pd.DataFrame(y_pred.numpy(), columns= ['score'])
    file_name = "y_pred/Data_Challenge_"+str(name)+".csv"
    results.to_csv(file_name, header = None, index = None)

    return y_pred, y_pred_probs
    

def print_cassif_report(Y_pred,Y_test):
    # Convert Y_pred to a DataFrame
    Y_pred_df = pd.DataFrame(Y_pred_tensor.numpy(), columns=['Predicted'])

    # Evaluate Y_pred compared to Y_test (assuming Y_test is a numpy array or a pandas Series)
    table = classification_report(Y_test, Y_pred_df['Predicted'])

    return table

#########################################################################
#     ENTRAINEMENT DU MODEL (AVEC MINI BATCH / SANS CROSS VALIDATION)
#########################################################################


def train_NN_with_custom_loss(model, optimizer, batch_size, X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor):

    # 1. Convertir les tensors en datasets puis en DataLoader pour gérer les mini-batchs
    train_dataset = TensorDataset(X_train_tensor, Y_train_one_hot, S_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = TensorDataset(X_test_tensor, Y_test_one_hot, S_test_tensor)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    
    # 2. Paramètres pour l'arrêt précoce
    # -----------------------------------
    patience = 5  # Nombre d'époques à attendre après la dernière amélioration de la perte de validation
    best_loss = None
    early_ending = None
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        
        model.train()
        train_loss = 0.0
        
        # 1/ exécuter les minibatches et recupérer la loss moyenne
        for X_batch, Y_batch, S_batch in train_loader:
            # Y_batch est one hot
            
            model.train()
            optimizer.zero_grad()
            outputs_train = model(X_batch)
            loss = soft_final_score_loss(Y_batch, outputs_train, S_batch)
            loss.backward()
            optimizer.step()

            # save mini-batch loss
            train_loss += loss.item()
        
        # Average loss pour l'epoch (après boucle mini-batchs)
        train_loss = train_loss / len(train_loader)       
        
        # 2. Vérifier si la perte de validation s'est améliorée (arret précoce)

        # Evaluation sur le jeu de données de test
        model.eval()
        test_loss = 0.0
        
        with torch.no_grad():
            for X_batch_test, Y_batch_test, S_batch_test in test_loader:
                outputs_test = model(X_batch_test)
                #Y_batch_test_one_hot = torch.nn.functional.one_hot(Y_batch_test, num_classes=Y_train.nunique())
                loss_test = soft_final_score_loss(Y_batch_test, outputs_test, S_batch_test)
                test_loss += loss_test.item()
                
        #average_test_loss = running_loss_test / len(test_loader)
        test_loss = test_loss / len(test_loader)
       
        # check if improvement in loss (compared to last epoch)
        if best_loss is None or test_loss < best_loss:
            best_loss = test_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f'Arrêt précoce après {epoch+1} époques')
                early_ending = epoch + 1
                break  # Arrêter l'entraînement
        
        # 3. Impression de l'apprentissage et des scores train et test
        if epoch==0 or (epoch+1) % 10 == 0:
            
            with torch.no_grad():
                
                # Calculate metrics for training data
                outputs_train = model(X_train_tensor) # probabilities
                # Evaluate predictions on training data
                final_score_train_ = get_final_score(Y_train_tensor, outputs_train, S_train_tensor)
                macro_f1_train = get_macro_f1(Y_train_tensor, outputs_train)
                inv_macro_tpr_gap_train = 1 - get_macro_tpr_gap(Y_train_tensor, outputs_train, S_train_tensor)
            
                # Calculate metrics for test data
                outputs_test = model(X_test_tensor)
                # Evaluate predictions on training data
                final_score_test_ = get_final_score(Y_test_tensor, outputs_test, S_test_tensor)
                macro_f1_test = get_macro_f1(Y_test_tensor, outputs_test)
                inv_macro_tpr_gap_test = 1 - get_macro_tpr_gap(Y_test_tensor, outputs_test, S_test_tensor)

                print(f'Epoch {epoch+1}, Loss: {loss.item()}, Final Score Train: {final_score_train_.item()}, Final Score Test: {final_score_test_.item()} (gap {final_score_test_-final_score_train_}) macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}, 1-TPR Gap Train: {inv_macro_tpr_gap_train}, 1-TPR Gap Test: {inv_macro_tpr_gap_test}')
            
    # 4. Make Predictions and Evaluate with final_score
    # -------------------------------------------------
            
    with torch.no_grad():
        model.eval()

        Y_train_pred_probs = model(X_train_tensor) # dim = 28 (Probabilities for each class)
        
        # # Y_train_pred_tensor = torch.argmax(Y_train_pred_probs, dim=1)  # dim = 1 (Get the class with the highest probability)
        final_score_train = get_final_score(Y_train_tensor, Y_train_pred_probs, S_train_tensor)

        Y_pred_probs = model(X_test_tensor) # dim = 28 (Probabilities for each class)
        Y_pred_tensor = torch.argmax(Y_pred_probs, dim=1)  # dim = 1 (Get the class with the highest probability)
        macro_f1 = get_macro_f1(Y_test_tensor, Y_pred_tensor)
        inv_macro_tpr_gap = 1 - get_macro_tpr_gap(Y_test_tensor, Y_pred_probs, S_test_tensor)
        final_score = get_final_score(Y_test_tensor, Y_pred_probs, S_test_tensor)
        
        print(f'Final Evaluation Score: {final_score.item()} gap {final_score.item()-final_score_train.item()} || Macro F1: {macro_f1.item()} 1-TPR_gap: { inv_macro_tpr_gap.item() }')

    return model, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap, early_ending,final_score_train


In [35]:
##############################################################
# train_test_split (np.arrays)
##############################################################

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, Y56_train, Y56_test = train_test_split(debiased_X, Y56, test_size=0.2, random_state=42)

Y_train = Y56_train % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_train = Y56_train//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)

Y_test = Y56_test % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_test = Y56_test//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)

# impression des dimensions
print('train:',X_train.shape,Y_train.shape,S_train.shape)
print('test:',X_test.shape,Y_test.shape, S_test.shape)

##############################################################
# 1. Transform DataFrames into Tensors
##############################################################

X_tensor = torch.tensor(X.values, dtype=torch.float32)
Y_tensor = torch.tensor(Y.values, dtype=torch.long)
S_tensor = torch.tensor(S.values, dtype=torch.long)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.long)
S_train_tensor = torch.tensor(S_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.long)
S_test_tensor = torch.tensor(S_test.values, dtype=torch.long)

Y_train_one_hot = torch.nn.functional.one_hot(Y_train_tensor, num_classes=Y_train.nunique())
Y_test_one_hot = torch.nn.functional.one_hot(Y_test_tensor, num_classes=Y_train.nunique())

X_test_true_tensor = torch.tensor(X_test_true.values, dtype=torch.float32)
#S_test_true_tensor = torch.tensor(S_test_true.values, dtype=torch.long)

# impression des dimensions
print('train_tensor:',X_train_tensor.shape,Y_train_tensor.shape,S_train_tensor.shape, type(X_train_tensor))
print('test_tensor:',X_test_tensor.shape,Y_test_tensor.shape, S_test_tensor.shape, type(X_test_tensor))
print('Y_train_one_hot:',Y_train_one_hot.shape, type(Y_train_one_hot))
print('X_test_true_tensor:',X_test_true_tensor.shape, type(X_test_true_tensor))

train: (22199, 768) (22199,) (22199,)
test: (5550, 768) (5550,) (5550,)
train_tensor: torch.Size([22199, 768]) torch.Size([22199]) torch.Size([22199]) <class 'torch.Tensor'>
test_tensor: torch.Size([5550, 768]) torch.Size([5550]) torch.Size([5550]) <class 'torch.Tensor'>
Y_train_one_hot: torch.Size([22199, 28]) <class 'torch.Tensor'>
X_test_true_tensor: torch.Size([11893, 768]) <class 'torch.Tensor'>


In [36]:
#################################################
#       TEST D'UN MODEL (ET DU CODE)
################################################


# 1. Define the model and optimizer and train
# --------------------------------------------------

model = nn.Sequential(
    nn.Linear(768, 28),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Linear(28, 28),
    nn.Softmax(dim=1),  # LogSoftmax for multi-class classification
    )  

batch_size = 512
learning_rate=0.001
optimizer = optim.Adagrad(model.parameters(), lr=learning_rate, weight_decay=1/0.2)
num_epochs = 1000

# 2. Train the model with the custom loss function final_eval
# -----------------------------------------------------------
name = 'NN-28-28_Adam'+'_lr_'+str(learning_rate)+'_batch_size_'+str(batch_size)
print('\n\n Starting to train model', name)
model_trained, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap, early_ending, final_score_train = train_NN_with_custom_loss(model,optim.Adam(model.parameters(), lr=learning_rate), batch_size, X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor)

# Res=pd.DataFrame(columns=['model','optimizer','lr','batch_size','early_ending', 'final_score_train','final_score','macro_f1','macro_tpr_gap'])
# Res.loc[i]=[name,optimizer,learning_rate,batch_size, early_ending,final_score_train, final_score, macro_f1, inv_macro_tpr_gap]

# Save predictions on X_test_true
save_Y_pred_tofile(X_test_true_tensor, model_trained,name)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)



 Starting to train model NN-28-28_Adam_lr_0.001_batch_size_512
Epoch 1, Loss: 0.9312233328819275, Final Score Train: 0.5614883899688721, Final Score Test: 0.561823844909668 (gap 0.00033545494079589844) macro F1 Train: 0.13548072378038073, macro F1 Test: 0.13768619321322936, 1-TPR Gap Train: 0.987496018409729, 1-TPR Gap Test: 0.9859615564346313
Epoch 10, Loss: 0.6553373336791992, Final Score Train: 0.7950546741485596, Final Score Test: 0.7695534229278564 (gap -0.025501251220703125) macro F1 Train: 0.6346785599846217, macro F1 Test: 0.5994005510131909, 1-TPR Gap Train: 0.9554307460784912, 1-TPR Gap Test: 0.9397063255310059
Epoch 20, Loss: 0.650231122970581, Final Score Train: 0.8250597715377808, Final Score Test: 0.776787519454956 (gap -0.04827225208282471) macro F1 Train: 0.6913052690432434, macro F1 Test: 0.6227293060857644, 1-TPR Gap Train: 0.9588142037391663, 1-TPR Gap Test: 0.9308457374572754
Epoch 30, Loss: 0.696786105632782, Final Score Train: 0.8408527374267578, Final Score Tes

**"SENSITIVE FRONTIER" MODELS <br>(USED ON DEBIASED DATASET)**
---
K nearest neighboors<br>
SVM linear<br>
SVM gaussian kernel<br>
One versus all

**LINEAR REGRESSION OF MODELS**
---


In [52]:
list = [ '0_LogReg_baseline' , 
        '1_LogReg_optimised' , 
        '2_Reglog_StratKFold',
        '3_LogReg_optimised_orthogonal 78,08%' , 
        '4_Reglog_DUAL_orthogonal_FxH',
        '4_Reglog_DUAL_orthogonal_F',
        '4_Reglog_DUAL_orthogonal_H',
        '5_Reglog56_StratKFold_Orthogonal_CustomLoss',
        '6_NN-28-28_Adam_lr_0.001_batch_size_512'
        ]

with open(path_model + list[0] + '.pkl', 'rb') as f: clf_0 = pickle.load(f)
with open(path_model + list[1] + '.pkl', 'rb') as f: clf_1 = pickle.load(f)
with open(path_model + list[2] + '.pkl', 'rb') as f: clf_2 = pickle.load(f)
with open(path_model + list[3] + '.pkl', 'rb') as f: clf_3 = pickle.load(f)
with open(path_model + list[4] + '.pkl', 'rb') as f: clf_4 = pickle.load(f)
with open(path_model + list[5] + '.pkl', 'rb') as f: clf_4F = pickle.load(f)
with open(path_model + list[6] + '.pkl', 'rb') as f: clf_4H = pickle.load(f)
with open(path_model + list[7] + '.pkl', 'rb') as f: clf_5 = pickle.load(f)
with open(path_model + list[8] + '.pkl', 'rb') as f: clf_6 = pickle.load(f)
# with open(path_model + list[7] + '.pkl', 'rb') as f: clf_7 = pickle.load(f)

In [99]:
# Generate a dataframe will all predictions for X

predictions = []

for clf in [ clf_3,clf_5] : # clf1 and 2 are identical
    
    preds = clf.predict(debiased_X)
    print( clf,preds.shape)
    predictions.append(preds)

# Ajout mannuel du prédicteur dual (clf_4)
# Predict model on test_true
Y_pred_F = model_F.predict(debiased_X_F)
Y_pred_H = model_H.predict(debiased_X_H)
# compute final result depending on S_test_true
Y_pred_4 = np.zeros(len(X))
Y_pred_4[ S == 1 ] = Y_pred_F
Y_pred_4[ S == 0 ] = Y_pred_H
predictions.append(Y_pred_4)



#Ajout du NN
output_6_NN = model_trained(X_tensor)
output_6_labels = torch.argmax(output_6_NN, dim=1)
predictions.append(output_6_labels)

# Ajout de Y et S pour analyse
predictions.append(S)
predictions.append(Y)

X_predictions = np.column_stack(predictions)
X_predictions[:, 1] = X_predictions[:, 1] % 28  # corriger Y56

print(X_predictions.shape)
X_predictions[:20,:]

#-----------------------------------
#   prediction
#-----------------------------------

from scipy.stats import mode
# Trouver le mode dans chaque ligne
modes, counts = mode(X_predictions[:,:-2], axis=1)  # ne pas prendre les 2 dernières colonnes Y et S

# Le mode est retourné dans un array; nous le simplifions pour un usage direct
Y_pred = modes.flatten()

get_scores(Y_pred,Y,S)
# modes_simplified.to_csv(path_Y_pred_true + "Data_Challenge_GLOBAL.csv", header = None, index = None)
X_predictions[:20,:]


LogisticRegression(C=0.2, max_iter=5000, random_state=42, verbose=1) (27749,)
LogisticRegression(C=0.2, max_iter=5000, random_state=42) (27749,)
(27749, 6)
Accuracy on transformed test data: 0.8590940214061767
final score : 0.870375472870099
macro_f1    : 0.8345467456152175
inv_macro_gap 0.9062042001249804


array([[ 6.,  6.,  6.,  6.,  0.,  6.],
       [ 9.,  9.,  9.,  9.,  1.,  9.],
       [ 6.,  6.,  6.,  6.,  0.,  6.],
       [22., 22., 22., 22.,  1., 22.],
       [21., 21., 21., 21.,  0., 21.],
       [25., 19., 19., 25.,  1., 19.],
       [18., 18., 18., 18.,  0., 18.],
       [13., 13., 13., 13.,  1., 13.],
       [18., 18., 18., 18.,  1., 18.],
       [19., 19., 19., 19.,  0., 19.],
       [ 6.,  6.,  6.,  6.,  1.,  6.],
       [ 2.,  2.,  2.,  2.,  0.,  2.],
       [21., 21., 21., 21.,  0., 21.],
       [ 9.,  9.,  9., 21.,  0.,  9.],
       [21., 21., 21., 21.,  0., 21.],
       [ 1.,  1.,  1.,  1.,  0.,  1.],
       [26.,  1., 26., 26.,  1., 26.],
       [26., 26., 26., 26.,  1., 12.],
       [26., 21., 26., 26.,  1., 26.],
       [19., 25., 25., 19.,  0., 11.]])

In [101]:
# Generate a dataframe will all predictions for X_test_true



predictions = []

for clf in [ clf_3,clf_5] :  # clf1 and 2 are identical
    
    preds = clf.predict(debiased_X_test_true)
    print( clf,preds.shape)
    predictions.append(preds)

# Ajout mannuel du prédicteur dual (clf_4)
# Predict model on test_true
Y_pred_F = model_F.predict(debiased_X_test_true_F)
Y_pred_H = model_H.predict(debiased_X_test_true_H)
# compute final result depending on S_test_true
Y_pred_4 = np.zeros(len(X_test_true))
Y_pred_4[ S_test_true == 1 ] = Y_pred_F
Y_pred_4[ S_test_true == 0 ] = Y_pred_H
predictions.append(Y_pred_4)

#Ajout du NN
output_6_NN = model_trained(X_test_true_tensor)
output_6_labels = torch.argmax(output_6_NN, dim=1)
predictions.append(output_6_labels)

# Ajout de S #+1 si femme et - 1 si homme (S - 0.5)*2
predictions.append(S_test_true)

# Ajout de Y pour comparer
#predictions.append(Y)

# Ajuster les valeurs pour la rgression sur Y56
X_predictions = np.column_stack(predictions)
X_predictions[:, 1] = X_predictions[:, 1] % 28

print(X_predictions.shape)
X_predictions[:20,:]

#-----------------------------------
#   prediction
#-----------------------------------

from scipy.stats import mode
# Trouver le mode dans chaque ligne
modes, counts = mode(X_predictions[:,:-1], axis=1)

# Le mode est retourné dans un array; nous le simplifions pour un usage direct
Y_true_pred = modes.flatten().astype(int)
Y_true_pred = pd.DataFrame(Y_true_pred)
Y_true_pred.to_csv(path_Y_pred_true + "Data_Challenge_GLOBAL.csv", header = None, index = None)

X_predictions[:20,:]
#X_predictions[:20,:-1]


LogisticRegression(C=0.2, max_iter=5000, random_state=42, verbose=1) (11893,)
LogisticRegression(C=0.2, max_iter=5000, random_state=42) (11893,)
(11893, 5)


array([[18., 18., 18., 18.,  0.],
       [21., 21., 21., 21.,  0.],
       [18., 12., 18., 18.,  0.],
       [ 2.,  2.,  2.,  2.,  0.],
       [21., 21., 13., 21.,  1.],
       [ 4.,  4.,  4.,  4.,  0.],
       [18., 13., 13., 12.,  1.],
       [18., 18., 18., 18.,  0.],
       [18., 11., 18., 18.,  0.],
       [19., 19., 19., 19.,  1.],
       [13., 13., 13., 13.,  0.],
       [26.,  5.,  5., 24.,  0.],
       [21., 21., 21., 21.,  0.],
       [19., 19., 19., 19.,  0.],
       [11., 11., 11., 11.,  1.],
       [21., 21., 21., 21.,  0.],
       [11., 11., 11., 11.,  1.],
       [21., 21., 21., 21.,  1.],
       [21., 21., 21., 21.,  0.],
       [18., 18., 18., 18.,  0.]])

**SCORES A BLANC**
---

In [ ]:
#####################################################
#   SCORE A BLANC
#####################################################


n=Y.shape[0]
Y_pred =np.ones(n)*1
accuracy= accuracy_score(Y, Y_pred)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy}")
eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2

#print results
print('final score',final_score)
print('macro_fscore',eval_scores['macro_fscore'])
print('1-eval_scores[\'TPR_GAP\']',1-eval_scores['TPR_GAP'])

In [ ]:
# test score "prediction uniforme"

n=Y.shape[0]

Scores_U=pd.DataFrame(columns=['N','N_f','N_h','accuracy','final_score','macro_f1','macro_gap'])

print(Scores_U)
for i in range(28):
    #Test value for all i values
    test=pd.DataFrame(np.ones(11893,dtype=int)*i)
    test.to_csv("all/Data_Challenge_all_"+str(i)+".csv", header = None, index = None)
    
    Y_pred=pd.DataFrame(np.ones(n,dtype=int)*i)
    accuracy= accuracy_score(Y, Y_pred)  # Y_test are your original test labels
    eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
    macro_f1 = eval_scores['macro_fscore']
    macro_gap = eval_scores['TPR_GAP']
    final_score = (macro_f1 +1- macro_gap)/2
    # check number of occurence
    N = (Y==i).sum()
    N_f = (Y56 == i + 28).sum()
    N_h = (Y56 == i).sum()
    Scores_U.loc[i]= [N, N_f, N_h, accuracy,final_score,macro_f1,macro_gap]

Scores_U

In [ ]:
size = Scores_U['N']
score = Scores_U['final_score']


# Créer et entraîner le modèle de régression linéaire
from sklearn.linear_model import LinearRegression
reverse = LinearRegression()
reverse.fit(np.array(score).reshape(-1, 1),np.array(size).reshape(-1, 1))

# Load the scores for X_true
scores_true = pd. read_csv('all/distribution_Y_true.txt',header = None,index_col=0)
dist_true_pred = reverse.predict(scores_true) #/ 11893 * 27749

# Afficher la distribution de X_test_true
plt.scatter(size,score)
plt.scatter(dist_true_pred,scores_true)
plt.show

In [ ]:
plt.scatter(Scores_U['final_score'],scores_true)

In [ ]:
plt.plot(range(28),size,label='X')
plt.plot(range(28),np.round(dist_true_pred),'X_test_true')
plt.legend()

In [ ]:
# Normalize data (L2 norm recommended for embeddings)
#X = normalize(X, norm='l2')
#X_test_true = normalize(X_test_true, norm='l2')



In [ ]:
##############################################################
# train_test_split with Y56 (np.arrays)
##############################################################

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, Y56_train, Y56_test = train_test_split(X, Y56, test_size=0.2, random_state=42)
Y_train = Y56_train % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_train = Y56_train//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)
Y_test = Y56_test % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_test = Y56_test//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)

# impression des dimensions
print('train:',X_train.shape,Y_train.shape,S_train.shape)
print('test:',X_test.shape,Y_test.shape, S_test.shape)

**Explore data**
---

In [ ]:
dist = pd.DataFrame({'label':Y,'S':S})

# Sum counts of 1s and 0s to get the total count for each label
result = dist.groupby('label')['S'].value_counts().unstack(fill_value=0)
result.columns = ['S', 'not_S']
result['total_count'] = result.sum(axis=1)

# Calculate totals for each column
totals = result.sum(axis=0)
result.loc[30] = totals

# Calculate total count percentages for each count
result['%_S_label'] = round((result['S'] / result['total_count']) * 100)
result['%_not_S_label'] = round((result['not_S'] / result['total_count']) * 100)
result['%_total'] = np.round(result['total_count']/len(Y)*100,2 ) # % of total count
result['%_S_total'] = np.round(result['S']/(S==1).sum()*100,2 ) # % of total count
result['%_not_S_total'] = np.round(result['not_S']/(S!=1).sum()*100,2 ) # % of total count
result['diff_%_S_total']=result['%_S_total']-result['%_total']
result['|']='|'
#Reorder table
result = result.reindex(columns=['total_count', '%_total','|','S','%_S_total','diff_%_S_total','|','not_S', '%_not_S_total','|','%_S_label', '%_not_S_label'])


result

In [ ]:
# Extract the data from the '%_S' column
labels = pd.to_numeric(result.index)
data = result['%_total']
data_S = result['%_S_total']
data_not_S = result['%_not_S_total']

# Create a bar plot
plt.plot(labels[:-1], data[:-1])
plt.plot(labels[:-1],data_S[:-1], label='Sensitive')
plt.plot(labels[:-1],data_not_S[:-1], label='Not_sensitive')

# Add labels and title
plt.xlabel('Label')
plt.ylabel('Percentage of total')
plt.title('Percentage of total for sample, S and non-S')
plt.legend()
# Show the plot
plt.show()

In [ ]:
result_sorted = result.iloc[:-1].sort_values(by='diff_%_S_total', ascending=False)
result_sorted['original_label'] = result_sorted.index
result_sorted=result_sorted.reset_index(drop=True)
#result_sorted.reset_index(drop=True, inplace=True)
result_sorted

In [ ]:
# Extract the data from the '%_S' column
labels_sorted = result_sorted['original_label']
#print(labels_sorted)
data = result_sorted['%_total']
data_S = result_sorted['%_S_total']
data_not_S = result_sorted['%_not_S_total']

# Create a bar plot
plt.plot(data)
plt.plot(data_S, label='Sensitive')
plt.plot(data_not_S, label='Not_sensitive')

# Add labels and title
plt.xlabel('Label')
plt.ylabel('Percentage of total')
#plt.xticks(labels_sorted)
plt.xticks(ticks=range(len(labels_sorted)), labels=labels_sorted, rotation=90)  # Rotate if there are many labels

plt.title('Percentage of total for sample, S and non-S')
plt.legend()
# Show the plot
plt.show()

**SECOND METHOD - ADVERSARIAL NN**
---

In [ ]:
# Refresh training data
# X_train, X_test, Y_train, Y_test, S_train, S_test = X_train_, X_test_, Y_train_, Y_test_, S_train_, S_test_

In [106]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy , CategoricalCrossentropy

from tensorflow.keras.metrics import AUC
from tensorflow.keras.utils import to_categorical

# Assuming X_train is your input embeddings and S is your sensitive attribute

# Input layer
input_layer = Input(shape=(768,))

# Main task classifier layers
main_task_hidden = Dense(256, activation='relu')(input_layer)
main_task_output = Dropout(0.5)(main_task_hidden)
main_task_output = Dense(28, activation='softmax', name='main_task_output')(main_task_hidden)

# Adversarial component layers
adversary_hidden = Dense(256, activation='relu')(main_task_hidden)
adversary_hidden = Dropout(0.5)(adversary_hidden)
adversarial_output = Dense(1, activation='sigmoid', name='adversarial_output')(adversary_hidden)

# Model
model_2 = Model(inputs=input_layer, outputs=[main_task_output, adversarial_output])

# Optimizers
#main_task_optimizer = Adam(learning_rate=0.001)
#adversarial_optimizer = Adam(learning_rate=0.001)

# Loss functions
main_task_loss = CategoricalCrossentropy()
adversarial_loss = BinaryCrossentropy()

# Compile the model
model_2.compile(optimizer=Adam(learning_rate=0.001),
              loss={'main_task_output': main_task_loss, 'adversarial_output': adversarial_loss},
              loss_weights={'main_task_output':1., 'adversarial_output':-0.1},
              metrics={'main_task_output': ['accuracy'], 'adversarial_output': [AUC()]})

# Prepare the labels for the main task and the adversarial task
Y_main_task = to_categorical(Y_train, num_classes=28)#Y_train # Your main task labels
Y_adversary = S_train    # Your sensitive attribute labels

# check size on input .output
print(X_train.shape,Y_main_task.shape,Y_adversary.shape)

# X normal:isation
# none in method 2

# Train the model
model_2.fit(X_train, {'main_task_output': Y_main_task, 'adversarial_output': Y_adversary}, epochs=10)

# After training, you can use the output of `main_task_hidden` as your new unbiased representation


(22199, 768) (22199, 28) (22199,)
Epoch 1/10


694/694 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - adversarial_output_auc_3: 0.5019 - loss: -32135.4043 - main_task_output_accuracy: 0.3827
Epoch 2/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - adversarial_output_auc_3: 0.5000 - loss: -1229957.5000 - main_task_output_accuracy: 0.1678
Epoch 3/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - adversarial_output_auc_3: 0.5000 - loss: -6222336.0000 - main_task_output_accuracy: 0.1413
Epoch 4/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - adversarial_output_auc_3: 0.5000 - loss: -16569529.0000 - main_task_output_accuracy: 0.1412
Epoch 5/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - adversarial_output_auc_3: 0.5000 - loss: -33675884.0000 - main_task_output_accuracy: 0.1342
Epoch 6/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - adversarial_output_auc_3: 0.5000 - loss: -57282136.0000 - main_task_output_accuracy: 0.1351
Epoch 7/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - adversarial_output_auc_3: 0.5000 - loss: -88983400.0000 - main_task_output_accuracy: 0

In [103]:
# Step 1: Transform X_train and X_test
feature_extractor_2 = Model(inputs=model_2.input, outputs=main_task_hidden)
X_train_transformed_2 = feature_extractor_2.predict(X_train)
X_test_transformed_2 = feature_extractor_2.predict(X_test)
X_test_true_transformed_2 = feature_extractor_2.predict(X_test_true)

# Step 2: Train a new classifier on the transformed training data
clf_2 = LogisticRegression(random_state=42, max_iter=5000,verbose=0,penalty='l2', C=0.2, tol=0.0001)  # Increase max_iter if needed for convergence
history_new_2 = clf_2.fit(X_train_transformed_2, Y_train)  # Y_train are your original training labels

# Step 3: Predict on the transformed test data and evaluate
Y_pred_2 = clf_2.predict(X_test_transformed_2)
accuracy_2= accuracy_score(Y_test, Y_pred_2)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_2}")

# Step 4 : Predict with gloabl score
eval_scores_2, confusion_matrices_eval_2 = gap_eval_scores(Y_pred_2, Y_test, S_test, metrics=['TPR'])
final_score_2 = (eval_scores_2['macro_fscore']+ (1-eval_scores_2['TPR_GAP']))/2
print('\nfinal',final_score_2)
print('macro_fscore',eval_scores_2['macro_fscore'])
print('1-eval_scores',1-eval_scores_2['TPR_GAP'])

694/694 ━━━━━━━━━━━━━━━━━━━━ 1s 831us/step
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 845us/step
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 792us/step


KeyboardInterrupt: 

In [ ]:
# Classify the provided test data with you classifier
y_test = clf_2.predict(X_test_true_transformed_2)
results=pd.DataFrame(y_test, columns= ['score'])

results.to_csv("Data_Challenge_Adversarial_NN.csv", header = None, index = None)
# np.savetxt('y_test_challenge_student.txt', y_test, delimiter=',')

**THIRD METHOD**
---

In [ ]:
from sklearn.preprocessing import StandardScaler

# get X_train an
# X_train, X_test, Y_train, Y_test, S_train, S_test = train_test_split(X, Y, S, test_size=0.3, random_state=42)

# Initialize StandardScaler
scaler = StandardScaler()

# Fit the scaler to your training data
scaler.fit(X_train)

# Transform your training data
X_train_standardized = scaler.transform(X_train)
X_test_standardized = scaler.transform(X_test)
X_test_true_standardized = scaler.transform(X_test_true)



In [108]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy , CategoricalCrossentropy

from tensorflow.keras.metrics import AUC
from tensorflow.keras.utils import to_categorical

# Assuming X_train is your input embeddings and S is your sensitive attribute

# Input layer
input_layer = Input(shape=(768,))

# Main task classifier layers
main_task_hidden = Dense(256, activation='relu')(input_layer)
main_task_hidden = Dropout(0.5)(main_task_hidden)
main_task_output = Dense(28, activation='softmax', name='main_task_output')(main_task_hidden)

# Adversarial component layers
adversary_hidden = Dense(256, activation='relu')(main_task_hidden)
adversary_hidden = Dropout(0.5)(adversary_hidden)
adversarial_output = Dense(1, activation='sigmoid', name='adversarial_output')(adversary_hidden)

# Model
model_3 = Model(inputs=input_layer, outputs=[main_task_output, adversarial_output])

# Optimizers
#main_task_optimizer = Adam(learning_rate=0.001)
#adversarial_optimizer = Adam(learning_rate=0.001)

# Loss functions
main_task_loss = CategoricalCrossentropy()
adversarial_loss = BinaryCrossentropy()

# Compile the model
model_3.compile(optimizer=Adam(learning_rate=0.001),
              loss={'main_task_output': main_task_loss, 'adversarial_output': adversarial_loss},
              metrics={'main_task_output': ['accuracy'], 'adversarial_output': [AUC()]})

# Prepare the labels for the main task and the adversarial task
Y_main_task = to_categorical(Y_train, num_classes=28)#Y_train # Your main task labels
Y_adversary = S_train    # Your sensitive attribute labels

# check size on input .output
print(X_train.shape,Y_main_task.shape,Y_adversary.shape)

# Train the model
model_3.fit(X_train, {'main_task_output': Y_main_task, 'adversarial_output': Y_adversary}, epochs=10)

# After training, you can use the output of `main_task_hidden` as your new unbiased representation


(22199, 768) (22199, 28) (22199,)
Epoch 1/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - adversarial_output_auc_5: 0.5372 - loss: 2.4315 - main_task_output_accuracy: 0.5348
Epoch 2/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - adversarial_output_auc_5: 0.7183 - loss: 1.5883 - main_task_output_accuracy: 0.7021
Epoch 3/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - adversarial_output_auc_5: 0.8276 - loss: 1.4163 - main_task_output_accuracy: 0.7226
Epoch 4/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - adversarial_output_auc_5: 0.8988 - loss: 1.2303 - main_task_output_accuracy: 0.7462
Epoch 5/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - adversarial_output_auc_5: 0.9271 - loss: 1.1553 - main_task_output_accuracy: 0.7520
Epoch 6/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - adversarial_output_auc_5: 0.9512 - loss: 1.0795 - main_task_output_accuracy: 0.7511
Epoch 7/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - adversarial_output_auc_5: 0.9569 - loss: 1.0326 - main_task_output_accuracy: 0.7607


In [110]:
# Step 1: Transform X_train and X_test
feature_extractor_3 = Model(inputs=model_3.input, outputs=main_task_hidden)
X_train_transformed_3 = feature_extractor_3.predict(X_train)
X_test_transformed_3 = feature_extractor_3.predict(X_test)
X_test_true_transformed_3 = feature_extractor_3.predict(X_test_true)

# Step 2: Train a new classifier on the transformed training data
new_classifier_3 = LogisticRegression(max_iter=10000)  # Increase max_iter if needed for convergence
history_new_3 = new_classifier_3.fit(X_train_transformed_3, Y_train)  # Y_train are your original training labels

# Step 3: Predict on the transformed test data and evaluate
Y_pred_3 = new_classifier_3.predict(X_test_transformed_3)
accuracy_3= accuracy_score(Y_test, Y_pred_3)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_3}")

  1/694 ━━━━━━━━━━━━━━━━━━━━ 17s 25ms/step

694/694 ━━━━━━━━━━━━━━━━━━━━ 1s 892us/step
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step
Accuracy on transformed test data: 0.7727927927927928


In [111]:
# calculate final score
eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred_3, Y_test, S_test, metrics=['TPR'])
final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2

print('macro_fscore',eval_scores['macro_fscore'])
print('1-eval_scores',1-eval_scores['TPR_GAP'])
print('final score (average)',final_score)

macro_fscore 0.6712117457725401
1-eval_scores 0.7484002113701219
final score (average) 0.709805978571331


In [113]:
# Load the "true" test data
X_test_true = dat['X_test']
S_test_true = dat['S_test'] 

X_test_true_transformed_3 = feature_extractor_3.predict(X_test_true)

# Classify the provided test data with you classifier
y_test_true = new_classifier_3.predict(X_test_true_transformed_3)
results_3=pd.DataFrame(y_test_true, columns= ['score'])

results_3.to_csv("Data_Challenge_AdversarialNN_2.csv", header = None, index = None)
# np.savetxt('y_test_challenge_student.txt', y_test, delimiter=',')

  1/372 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step

372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 988us/step


**4th METHOD**
---

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy , CategoricalCrossentropy

from tensorflow.keras.metrics import AUC
from tensorflow.keras.utils import to_categorical

# Assuming X_train is your input embeddings and S is your sensitive attribute

# Input layer
input_layer = Input(shape=(768,))

# Main task classifier layers
main_task_hidden = Dense(512, activation='relu')(input_layer)
main_task_hidden = Dropout(0.5)(main_task_hidden)
main_task_output = Dense(28, activation='softmax', name='main_task_output')(main_task_hidden)

# Adversarial component layers
adversary_hidden = Dense(512, activation='relu')(main_task_hidden)
adversary_hidden = Dropout(0.5)(adversary_hidden)
adversarial_output = Dense(1, activation='sigmoid', name='adversarial_output')(adversary_hidden)

# Model
model_4 = Model(inputs=input_layer, outputs=[main_task_output, adversarial_output])

# Optimizers
#main_task_optimizer = Adam(learning_rate=0.001)
#adversarial_optimizer = Adam(learning_rate=0.001)

# Loss functions
main_task_loss = CategoricalCrossentropy()
adversarial_loss = BinaryCrossentropy()

# Compile the model
model_4.compile(optimizer=Adam(learning_rate=0.001),
              loss={'main_task_output': main_task_loss, 'adversarial_output': adversarial_loss},
              metrics={'main_task_output': ['accuracy'], 'adversarial_output': [AUC()]})

# Prepare the labels for the main task and the adversarial task
Y_main_task = to_categorical(Y_train, num_classes=28)#Y_train # Your main task labels
Y_adversary = S_train    # Your sensitive attribute labels

# check size on input .output
print(X_train_standardized.shape,Y_main_task.shape,Y_adversary.shape)

# Train the model
model_4.fit(X_train_standardized, {'main_task_output': Y_main_task, 'adversarial_output': Y_adversary}, epochs=10)

# After training, you can use the output of `main_task_hidden` as your new unbiased representation


In [ ]:
# Step 1: Transform X_train and X_test
feature_extractor_4 = Model(inputs=model_4.input, outputs=main_task_hidden)
X_train_transformed_4 = feature_extractor_4.predict(X_train_standardized)
X_test_transformed_4 = feature_extractor_4.predict(X_test_standardized)
X_test_true_transformed_4 = feature_extractor_4.predict(X_test_true_standardized)

# Step 2: Train a new classifier on the transformed training data
new_classifier_4 = LogisticRegression(max_iter=10000)  # Increase max_iter if needed for convergence
history_4 = new_classifier_4.fit(X_train_transformed_4, Y_train)  # Y_train are your original training labels

# Step 3: Predict on the transformed test data and evaluate
Y_pred_4 = new_classifier_4.predict(X_test_transformed_4)
accuracy_4 = accuracy_score(Y_test, Y_pred_4)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_4}")

In [ ]:
eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred_4, Y_test, S_test, metrics=['TPR'])
#eval_scores#eval_scores['macro_fscore']
#eval_scores['TPR_GAP']
final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2
print(final_score)

In [ ]:
# Load the "true" test data
X_test_true = dat['X_test']
S_test_true = dat['S_test'] 

X_test_true_transformed_4 = feature_extractor_4.predict(X_test_true)

# Classify the provided test data with you classifier
y_test_true = clf.predict(X_test_true_transformed_4)
results_4=pd.DataFrame(y_test_true, columns= ['score'])

results_4.to_csv("Data_Challenge_MDI_341_4.csv", header = None, index = None)
# np.savetxt('y_test_challenge_student.txt', y_test, delimiter=',')